## Examples of usage

In [1]:
import numpy as np
import random
import pandas as pd
import meliora.src.meliora.core as vt
from scipy.stats import norm

In [2]:
# Import data
test_data_main = pd.read_csv('../data/pd_test_data_main.csv')
test_data_add = pd.read_csv('../data/pd_test_data_period_2.csv')
test_pd_psi = pd.read_csv('../data/pd_bellini.csv')
german_data = pd.read_csv('../data/german_data.csv')
lgd_t_test = pd.read_csv('../data/lgd_t_test.csv')
pd_transition_matrix = pd.read_csv('../data/pd_transition_matrix.csv')
pd_calibration = pd.read_csv('../data/pd_calibration.csv')

In [3]:
test_data_main.head()

,ratings,default_flag,predicted_pd
0,B,0,0.137215
1,A,0,0.130365
2,C,0,0.188544
3,A,0,0.178812
4,C,0,0.135670


In [4]:
# Binomial test
vt.binomial_test(test_data_main, "ratings", "default_flag", "predicted_pd")

,Rating class,Predicted PD,Total count,Defaults,Actual Default Rate,p_value,Reject H0
0,A,0.103429,406,55,0.135468,0.023892,True
1,B,0.099524,480,41,0.085417,0.867441,False
2,C,0.095719,114,10,0.087719,0.660553,False


In [5]:
# Jeffrey's test
vt.jeffreys_test(test_data_main, 'ratings', 'default_flag', 'predicted_pd', 0.05)

,Rating class,Predicted PD,Total count,Defaults,Actual Default Rate,p_value,Reject H0
0,A,0.103429,406,55,0.135468,0.019959,True
1,B,0.099524,480,41,0.085417,0.849552,False
2,C,0.095719,114,10,0.087719,0.598649,False


In [6]:
# Brier score
vt.brier_score(test_data_main, "ratings", "default_flag", "predicted_pd")

0.0012895084997917349

In [7]:
# Hosmer-Lemeshow test
vt.hosmer_test(test_data_main, "ratings", "default_flag", "predicted_pd")

[0.1302500370047408, False]

In [8]:
# Spiegelhalter test
vt.spiegelhalter_test(test_data_main, "ratings", "default_flag", "predicted_pd")

(-0.6637590511485174, False)

In [9]:
# ROC test
vt.roc_auc(test_data_main, "default_flag", "predicted_pd")

0.5008547549702419

In [10]:
# Spearman correlation
x = [1,2,3,4,5]
y = [5,6,7,8,7]
vt.spearman_corr(x, y).correlation

0.8207826816681233

In [11]:
# Somers D
x = [0, 1, 1, 1, 1] 
y = [1, 1, 1, 0, 1]
vt.somersd(x, y)

SomersDResult(statistic=-0.25, pvalue=0.3613104285261787, table=array([[0, 1],
       [1, 3]]))

In [12]:
# Kendall tau
x = [1, 2, 3, 2, 1, 3, 4, 2, 5, 2, 6, 5, 5]
y = [5, 5, 6, 2, 1, 4, 4, 2, 1, 2, 1, 5, 5]

tau, pvalue = vt.kendall_tau(x, y)
tau

0.030306509211290782

In [13]:
# PSI test
test_pd_psi['remaining_mat' + "_bin"] = pd.cut(test_pd_psi['remaining_mat'], bins=10, labels=False)
test_pd_psi['year_bins'] = np.where(test_pd_psi['vintage_year'] < 2007, 'period_1', 'period_2')

# Export data
test_pd_psi.to_csv('test_pd_psi.csv', index=False)

# Show results
kala = vt.psi(test_pd_psi, 'year_bins', 'remaining_mat_bin')
kala[1]

1.0344129494141174

In [14]:
# IV
zen = vt.information_value(german_data, 'checkingstatus', 'GoodCredit')

zen[1]

0.6660115033513336

In [15]:
zen[0]

,Variable,Value,All,Bad,Share,Bad Rate,Distribution Good,Distribution Bad,WoE,IV
0,checkingstatus,A11,274,135,0.274,0.492701,0.198571,0.450000,-0.818099,0.205693
1,checkingstatus,A12,269,105,0.269,0.390335,0.234286,0.350000,-0.401392,0.046447
3,checkingstatus,A13,63,14,0.063,0.222222,0.070000,0.046667,0.405465,0.009461
2,checkingstatus,A14,394,46,0.394,0.116751,0.497143,0.153333,1.176263,0.404410


In [16]:
# Transform input data into the required format
df = test_data_main.groupby('ratings').agg({'predicted_pd': "mean", 'default_flag': ["count", "sum", "mean"]})
df.columns = ["PD", "N", "D", "Default Rate"]

realised_values = df["Default Rate"]
predicted_values = df["PD"]

# Calculate mean squared error
errors = realised_values - predicted_values
mse = (errors**2).sum() / len(errors)

# # Calculate null expectation and variance of MSE
expectations = sum(predicted_values * (1 - predicted_values)) / len(realised_values)
variances = sum(predicted_values * (1 - 2 * predicted_values)**2 * (1 - predicted_values)) / len(realised_values)**2

# Calculate standardized statistic
z_score = (mse - expectations) / np.sqrt(variances)  # todo: check formula

# Calculate standardized MSE as test statistic, then its p-value
outcome = z_score > norm.ppf(1 - 0.05/2)

In [17]:
errors

ratings
A    0.032039
B   -0.014107
C   -0.008000
dtype: float64

In [18]:
mse

0.00042983616659724496

In [19]:
norm.ppf(1 - 0.05/2)

1.959963984540054

In [20]:
# LGD t test
lgd_t_test.head()

,facility,predicted_lgd,realised_lgd,segment
0,1,0.74,0.75,7
1,2,0.30,0.39,3
2,3,0.76,0.67,7
3,4,0.64,0.66,6
4,5,0.41,0.43,4


In [21]:
df = vt.lgd_t_test(lgd_t_test, 
                                'predicted_lgd', 
                                'realised_lgd',
                                level="segment",
                                segment_col='segment'
                               ).sort_values(by='segment')
df

,segment,N,realised_lgd_mean,pred_lgd_mean,s2,mean_error,t_stat,p_value
8,0,110,0.043636,0.052545,0.002560,-0.008909,-1.846661,0.966245
6,1,89,0.147079,0.158652,0.003443,-0.011573,-1.860702,0.966937
9,2,95,0.245263,0.238211,0.003032,0.007053,1.248457,0.107482
1,3,99,0.341111,0.337273,0.003748,0.003838,0.623798,0.267105
3,4,106,0.448774,0.450189,0.003056,-0.001415,-0.263546,0.603677
7,5,112,0.545357,0.549821,0.003695,-0.004464,-0.777216,0.780658
2,6,95,0.640526,0.635053,0.003565,0.005474,0.893477,0.186942
0,7,87,0.745172,0.737356,0.003182,0.007816,1.292329,0.099853
5,8,105,0.842000,0.843048,0.003442,-0.001048,-0.182971,0.572412
4,9,102,0.943529,0.932647,0.003002,0.010882,2.005877,0.023772


In [22]:
df['p_value'].sum()

4.575081449451153

In [23]:
# Migration matrix stats
vt.migration_matrices_statistics(pd_transition_matrix, 'period_1_ratings', 'period_2_ratings')

(0.43581081081081086, 0.8108108108108109)

In [24]:
# Bayesian error rate
vt.bayesian_error_rate(test_data_main, "default_flag", "predicted_pd")

0.106

In [25]:
# cier
vt.cier(pd_calibration, "rating", "realised_pd", "count")

0.024548595310375846

In [27]:
# cier
vt.kullback_leibler_dist(pd_calibration, "rating", "realised_pd", "count")

0.006240325352140225

#### Relevant links
- https://machinelearningmastery.com/divergence-between-probability-distributions/
- https://medium.com/@monadsblog/the-kullback-leibler-divergence-5071c707a4a6
- https://documentation.sas.com/doc/en/pgmsascdc/v_011/statug/statug_code_logiex19.htm